Last Update @ 2020.12.04

- Huggingface Transformers 4.0.0  버전 반영

# Package 설치 & 데이터 받기

In [ ]:
!pip install -q transformers pytorch_lightning emoji soynlp

     |████████████████████████████████| 1.5MB 14.7MB/s 
     |████████████████████████████████| 675kB 53.8MB/s 
     |████████████████████████████████| 51kB 6.9MB/s 
     |████████████████████████████████| 419kB 46.8MB/s 
     |████████████████████████████████| 2.9MB 35.3MB/s 
     |████████████████████████████████| 890kB 50.6MB/s 
     |████████████████████████████████| 102kB 10.8MB/s 
     |████████████████████████████████| 276kB 49.5MB/s 
     |████████████████████████████████| 829kB 43.4MB/s 


In [ ]:
!git clone https://github.com/e9t/nsmc

Cloning into 'nsmc'...
remote: Enumerating objects: 14763, done.
remote: Total 14763 (delta 0), reused 0 (delta 0), pack-reused 14763
Receiving objects: 100% (14763/14763), 56.19 MiB | 13.73 MiB/s, done.
Resolving deltas: 100% (1749/1749), done.
Checking out files: 100% (14737/14737), done.


In [ ]:
!head nsmc/ratings_train.txt

id	document	label
9976970	아 더빙.. 진짜 짜증나네요 목소리	0
3819312	흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나	1
10265843	너무재밓었다그래서보는것을추천한다	0
9045019	교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정	0
6483659	사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다	1
5403919	막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움.	0
7797314	원작의 긴장감을 제대로 살려내지못했다.	0
9443947	별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네	0
7156791	액션이 없는데도 재미 있는 몇안되는 영화	1


# 패키지 import & 기본 Args 설정

In [ ]:
import os
import pandas as pd

from pprint import pprint

import torch
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.optim.lr_scheduler import ExponentialLR

from pytorch_lightning import LightningModule, Trainer, seed_everything

from transformers import BertForSequenceClassification, BertTokenizer, AdamW

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

import re
import emoji
from soynlp.normalizer import repeat_normalize

## 기본 학습 Arguments

In [ ]:
class Arg:
    random_seed: int = 42  # Random Seed
    pretrained_model: str = 'beomi/kcbert-large'  # Transformers PLM name
    pretrained_tokenizer: str = ''  # Optional, Transformers Tokenizer Name. Overrides `pretrained_model`
    auto_batch_size: str = 'power'  # Let PyTorch Lightening find the best batch size 
    batch_size: int = 0  # Optional, Train/Eval Batch Size. Overrides `auto_batch_size` 
    lr: float = 5e-6  # Starting Learning Rate
    epochs: int = 20  # Max Epochs
    max_length: int = 150  # Max Length input size
    report_cycle: int = 100  # Report (Train Metrics) Cycle
    train_data_path: str = "nsmc/ratings_train.txt"  # Train Dataset file 
    val_data_path: str = "nsmc/ratings_test.txt"  # Validation Dataset file 
    cpu_workers: int = os.cpu_count()  # Multi cpu workers
    test_mode: bool = False  # Test Mode enables `fast_dev_run`
    optimizer: str = 'AdamW'  # AdamW vs AdamP
    lr_scheduler: str = 'exp'  # ExponentialLR vs CosineAnnealingWarmRestarts
    fp16: bool = False  # Enable train on FP16
    tpu_cores: int = 0  # Enable TPU with 1 core or 8 cores

args = Arg()

## 기본값을 Override 하고싶은 경우 아래와 같이 수정

In [ ]:
!nvidia-smi

Sun Dec 20 15:03:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    25W / 300W |     10MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

위에서 GPU가 V100/P100이면 아래 `batch_size`  를 32 이상으로 하셔도 됩니다.

In [ ]:
#args.tpu_cores = 8  # Enables TPU
args.fp16 = True  # Enables GPU FP16
args.batch_size = 32  # Force setup batch_size

# Model 만들기 with Pytorch Lightning

In [ ]:
class Model(LightningModule):
    def __init__(self, options):
        super().__init__()
        self.args = options
        self.bert = BertForSequenceClassification.from_pretrained(self.args.pretrained_model)
        self.tokenizer = BertTokenizer.from_pretrained(
            self.args.pretrained_tokenizer
            if self.args.pretrained_tokenizer
            else self.args.pretrained_model
        )

    def forward(self, **kwargs):
        return self.bert(**kwargs)

    def training_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits
        
        preds = logits.argmax(dim=-1)

        y_true = labels.cpu().numpy()
        y_pred = preds.cpu().numpy()

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'train_loss': loss.cpu().detach().numpy().tolist(),
            'train_acc': metrics[0],
            'train_precision': metrics[1],
            'train_recall': metrics[2],
            'train_f1': metrics[3],
        }
        if (batch_idx % self.args.report_cycle) == 0:
            print()
            pprint(tensorboard_logs)
        return {'loss': loss, 'log': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        data, labels = batch
        output = self(input_ids=data, labels=labels)

        # Transformers 4.0.0+
        loss = output.loss
        logits = output.logits

        preds = logits.argmax(dim=-1)

        y_true = list(labels.cpu().numpy())
        y_pred = list(preds.cpu().numpy())

        return {
            'loss': loss,
            'y_true': y_true,
            'y_pred': y_pred,
        }

    def validation_epoch_end(self, outputs):
        loss = torch.tensor(0, dtype=torch.float)
        for i in outputs:
            loss += i['loss'].cpu().detach()
        _loss = loss / len(outputs)

        loss = float(_loss)
        y_true = []
        y_pred = []

        for i in outputs:
            y_true += i['y_true']
            y_pred += i['y_pred']

        # Acc, Precision, Recall, F1
        metrics = [
            metric(y_true=y_true, y_pred=y_pred)
            for metric in
            (accuracy_score, precision_score, recall_score, f1_score)
        ]

        tensorboard_logs = {
            'val_loss': loss,
            'val_acc': metrics[0],
            'val_precision': metrics[1],
            'val_recall': metrics[2],
            'val_f1': metrics[3],
        }

        print()
        pprint(tensorboard_logs)
        return {'loss': _loss, 'log': tensorboard_logs}

    def configure_optimizers(self):
        if self.args.optimizer == 'AdamW':
            optimizer = AdamW(self.parameters(), lr=self.args.lr)
        elif self.args.optimizer == 'AdamP':
            from adamp import AdamP
            optimizer = AdamP(self.parameters(), lr=self.args.lr)
        else:
            raise NotImplementedError('Only AdamW and AdamP is Supported!')
        if self.args.lr_scheduler == 'cos':
            scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=1, T_mult=2)
        elif self.args.lr_scheduler == 'exp':
            scheduler = ExponentialLR(optimizer, gamma=0.5)
        else:
            raise NotImplementedError('Only cos and exp lr scheduler is Supported!')
        return {
            'optimizer': optimizer,
            'scheduler': scheduler,
        }

    def read_data(self, path):
        if path.endswith('xlsx'):
            return pd.read_excel(path)
        elif path.endswith('csv'):
            return pd.read_csv(path)
        elif path.endswith('tsv') or path.endswith('txt'):
            return pd.read_csv(path, sep='\t')
        else:
            raise NotImplementedError('Only Excel(xlsx)/Csv/Tsv(txt) are Supported')

    def preprocess_dataframe(self, df):
        emojis = ''.join(emoji.UNICODE_EMOJI.keys())
        pattern = re.compile(f'[^ .,?!/@$%~％·∼()\x00-\x7Fㄱ-힣{emojis}]+')
        url_pattern = re.compile(
            r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)')

        def clean(x):
            x = pattern.sub(' ', x)
            x = url_pattern.sub('', x)
            x = x.strip()
            x = repeat_normalize(x, num_repeats=2)
            return x

        df['document'] = df['document'].map(lambda x: self.tokenizer.encode(
            clean(str(x)),
            padding='max_length',
            max_length=self.args.max_length,
            truncation=True,
        ))
        return df

    def train_dataloader(self):
        df = self.read_data(self.args.train_data_path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=True,
            num_workers=self.args.cpu_workers,
        )

    def val_dataloader(self):
        df = self.read_data(self.args.val_data_path)
        df = self.preprocess_dataframe(df)

        dataset = TensorDataset(
            torch.tensor(df['document'].to_list(), dtype=torch.long),
            torch.tensor(df['label'].to_list(), dtype=torch.long),
        )
        return DataLoader(
            dataset,
            batch_size=self.args.batch_size or self.batch_size,
            shuffle=False,
            num_workers=self.args.cpu_workers,
        )


In [ ]:
def main():
    print("Using PyTorch Ver", torch.__version__)
    print("Fix Seed:", args.random_seed)
    seed_everything(args.random_seed)
    model = Model(args)

    print(":: Start Training ::")
    trainer = Trainer(
        max_epochs=args.epochs,
        fast_dev_run=args.test_mode,
        num_sanity_val_steps=None if args.test_mode else 0,
        auto_scale_batch_size=args.auto_batch_size if args.auto_batch_size and not args.batch_size else False,
        # For GPU Setup
        deterministic=torch.cuda.is_available(),
        gpus=-1 if torch.cuda.is_available() else None,
        precision=16 if args.fp16 else 32,
        # For TPU Setup
        tpu_cores=args.tpu_cores if args.tpu_cores else None,
    )
    trainer.fit(model)

# 학습!

> 주의: 1epoch별로 GPU-P100기준 약 2~3시간, GPU V100기준 ~40분이 걸립니다.

> Update @ 2020.09.01
> 최근 Colab Pro에서 V100이 배정됩니다.

```python
# 1epoch 기준 아래 score가 나옵니다.
{'val_acc': 0.90522,
 'val_f1': 0.9049023739289227,
 'val_loss': 0.23429009318351746,
 'val_precision': 0.9143146796431468,
 'val_recall': 0.8956818813808446}
```

In [ ]:
main()

Using PyTorch Ver 1.7.0+cu101
Fix Seed: 42


Some weights of the model checkpoint at beomi/kcbert-large were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initial

/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:49: UserWarning: You have set progress_bar_refresh_rate < 20 on Google Colab. This may crash. Consider using progress_bar_refresh_rate >= 20 in Trainer.
  warnings.warn(*args, **kwargs)
GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


:: Start Training ::



  | Name | Type                          | Params
-------------------------------------------------------
0 | bert | BertForSequenceClassification | 334 M 
-------------------------------------------------------
334 M     Trainable params
0         Non-trainable params
334 M     Total params


...


{'val_acc': 0.90284,
 'val_f1': 0.9052984521813716,
 'val_loss': 0.2487463653087616,
 'val_precision': 0.8888038277511962,
 'val_recall': 0.9224168752234537}

{'train_acc': 0.90625,
 'train_f1': 0.8695652173913044,
 'train_loss': 0.15822789072990417,
 'train_precision': 0.7692307692307693,
 'train_recall': 1.0}

{'train_acc': 0.90625,
 'train_f1': 0.9268292682926829,
 'train_loss': 0.25143811106681824,
 'train_precision': 0.95,
 'train_recall': 0.9047619047619048}

{'train_acc': 0.875,
 'train_f1': 0.8749999999999999,
 'train_loss': 0.2285623997449875,
 'train_precision': 0.8235294117647058,
 'train_recall': 0.9333333333333333}

{'train_acc': 0.90625,
 'train_f1': 0.9090909090909091,
 'train_loss': 0.16720372438430786,
 'train_precision': 0.8823529411764706,
 'train_recall': 0.9375}

{'train_acc': 0.8125,
 'train_f1': 0.8333333333333333,
 'train_loss': 0.3446575403213501,
 'train_precision': 0.8823529411764706,
 'train_recall': 0.7894736842105263}

{'train_acc': 1.0,
 'train_f1': 1.0,


{'val_acc': 0.90228,
 'val_f1': 0.9047229047229046,
 'val_loss': 0.24928703904151917,
 'val_precision': 0.8885058791987437,
 'val_recall': 0.9215429229730266}

{'train_acc': 0.8125,
 'train_f1': 0.7692307692307692,
 'train_loss': 0.396221786737442,
 'train_precision': 0.6666666666666666,
 'train_recall': 0.9090909090909091}

{'train_acc': 0.875,
 'train_f1': 0.9,
 'train_loss': 0.35395970940589905,
 'train_precision': 0.8571428571428571,
 'train_recall': 0.9473684210526315}

{'train_acc': 0.90625,
 'train_f1': 0.9090909090909091,
 'train_loss': 0.20594477653503418,
 'train_precision': 0.9375,
 'train_recall': 0.8823529411764706}

{'train_acc': 0.84375,
 'train_f1': 0.8,
 'train_loss': 0.26712071895599365,
 'train_precision': 0.6666666666666666,
 'train_recall': 1.0}

{'train_acc': 0.96875,
 'train_f1': 0.975609756097561,
 'train_loss': 0.12301953136920929,
 'train_precision': 1.0,
 'train_recall': 0.9523809523809523}

{'train_acc': 0.90625,
 'train_f1': 0.888888888888889,
 'train_loss


{'val_acc': 0.90232,
 'val_f1': 0.9046354512438006,
 'val_loss': 0.2551332116127014,
 'val_precision': 0.8895587727045813,
 'val_recall': 0.9202319945973861}

{'train_acc': 0.90625,
 'train_f1': 0.8695652173913044,
 'train_loss': 0.30564478039741516,
 'train_precision': 0.7692307692307693,
 'train_recall': 1.0}

{'train_acc': 1.0,
 'train_f1': 1.0,
 'train_loss': 0.08250430971384048,
 'train_precision': 1.0,
 'train_recall': 1.0}

{'train_acc': 0.9375,
 'train_f1': 0.9411764705882353,
 'train_loss': 0.13801278173923492,
 'train_precision': 1.0,
 'train_recall': 0.8888888888888888}

{'train_acc': 0.9375,
 'train_f1': 0.9333333333333333,
 'train_loss': 0.23054976761341095,
 'train_precision': 1.0,
 'train_recall': 0.875}

{'train_acc': 0.90625,
 'train_f1': 0.8421052631578948,
 'train_loss': 0.21536357700824738,
 'train_precision': 0.8,
 'train_recall': 0.8888888888888888}

{'train_acc': 0.9375,
 'train_f1': 0.9333333333333333,
 'train_loss': 0.17817901074886322,
 'train_precision': 0.9


{'val_acc': 0.90392,
 'val_f1': 0.9051941900852541,
 'val_loss': 0.2450191080570221,
 'val_precision': 0.8994078199145065,
 'val_recall': 0.9110554959679021}

{'train_acc': 0.9375,
 'train_f1': 0.9411764705882353,
 'train_loss': 0.24791723489761353,
 'train_precision': 0.9411764705882353,
 'train_recall': 0.9411764705882353}

{'train_acc': 0.9375,
 'train_f1': 0.9166666666666666,
 'train_loss': 0.1707983911037445,
 'train_precision': 0.9166666666666666,
 'train_recall': 0.9166666666666666}

{'train_acc': 0.9375,
 'train_f1': 0.9375,
 'train_loss': 0.1909032166004181,
 'train_precision': 0.9375,
 'train_recall': 0.9375}

{'train_acc': 0.90625,
 'train_f1': 0.896551724137931,
 'train_loss': 0.19730786979198456,
 'train_precision': 1.0,
 'train_recall': 0.8125}

{'train_acc': 1.0,
 'train_f1': 1.0,
 'train_loss': 0.11435156315565109,
 'train_precision': 1.0,
 'train_recall': 1.0}

{'train_acc': 0.90625,
 'train_f1': 0.8695652173913043,
 'train_loss': 0.23448891937732697,
 'train_precisio


{'val_acc': 0.90222,
 'val_f1': 0.9055575946066026,
 'val_loss': 0.253519743680954,
 'val_precision': 0.8813642174926675,
 'val_recall': 0.931116672625432}



In [ ]:
import pandas as pd
ko = pd.read_csv("./ko_data.csv")
ko_data = [['Id', 'Predicted']]

for idx in range(len(ko['Sentence'])):
  sen = ko['Sentence'][idx]
  logit = test_sentences([sen])
  ko_data.append([idx, np.argmax(logit)])

dataframe = pd.DataFrame(ko_data)
dataframe.to_csv("test.csv", header=False, index=False)

In [ ]:
!pip install pyyaml h5py

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.0


In [ ]:
model = create_model()
!mkdir -p saved_model
model.save('my_model.h5') 

NameError: ignored